In [1]:
import numpy as np
from qiskit import *
import import_ipynb
import qkd_random_gen as qrg

importing Jupyter notebook from qkd_random_gen.ipynb


In [2]:
class Alice:
    def __init__(self, epsilon, no_of_children):
        self.epsilon = epsilon
        self.no_of_children = no_of_children
        self.measurements = list()
        self.__gen_bit_flip_index()
    
    def __gen_bit_flip_index(self):
        self.__index = 1 # np.random.randint(self.no_of_children) + 1
        
    def get_index(self)->int:
        return self.__index
        
    def measure(self, measurement: int)->None:
        self.measurements.append(measurement)
    
class Children:
    def __init__(self, epsilon):
        self.epsilon = epsilon
        self.flip = False
        self.bases = list()
        self.__measurements = list()
        
    def set_flip(self):
        self.flip = True
        
    def gen_measurement_basis(self, qc: QuantumCircuit, qr: int)->None:
        e = np.random.rand()
        if e <= self.epsilon:
            self.bases.append('X')
            qc.h(qr)
        else:
            self.bases.append('Z')
        qc.measure(qr, qr)
        
    def get_alice_measurements(self, alice_measurements)->None:
        self.alice_measurements = alice_measurements
        
    def get_all_bases(self, all_bases)->None:
        self.all_bases = all_bases 
        
    def gen_key(self):
        for i in range(len(self.alice_measurements)):
            if self.flip and self.alice_measurements[i] == 1:
                self.__measurements[i] = 1 - self.__measurements[i]
        idx_to_keep = list()
        for i in range(len(self.bases)):
            keep = True
            for bases in self.all_bases:
                if bases[i] == 'X':
                    keep = False
                    break
            if keep:
                idx_to_keep.append(i)
        self.__key = [self.__measurements[idx] for idx in idx_to_keep]
            
    def measure(self, measurement: int)->None:
        self.__measurements.append(measurement)

In [5]:
class CommonSpace:
    def __init__(self, no_of_children = 2, no_of_measurements = 1024):
        self.epsilon = self.set_epsilon()
        self.no_of_children = no_of_children
        self.no_of_measurements = no_of_measurements
        self.alice = Alice(self.epsilon, self.no_of_children)
        self.children = [Children(self.epsilon) for _ in range(self.no_of_children)]
        self.qc = list()
    
    def set_epsilon(self)->float:
        qbg = qrg.QBitGen()
        qbg.load_provider()
        qbg.init_num_circ()
        qbg.evaluate_circ(128, True)
        return qbg.get_random(True)
        
    def gen_entanglement(self)->QuantumCircuit:
        qc = QuantumCircuit(self.no_of_children + 1, self.no_of_children + 1)
        qc.h(0)
        qc.h(1)
        for i in range(1, self.no_of_children):
            qc.cx(i, i + 1)
        idx = self.alice.get_index()
        qc.cx(0, idx)
        qc.barrier()
        return qc
    
    def measurement_phase(self):
        for _ in range(self.no_of_measurements):
            qc = self.gen_entanglement()
            qc.measure(0, 0)
            self.__set_bases(qc)
            self.qc.append(qc)
            aer_sim = Aer.get_backend('aer_simulator')
            qobj = assemble(qc, shots = 1, memory=True)
            result = aer_sim.run(qobj).result()
            self.alice.measure(int(result.get_memory()[0][-1]))
            for i in range(self.no_of_children):
                self.children[i].measure(int(result.get_memory()[0][-2 - i]))
                
    def agreement_phase(self):
        all_bases = list()
        for child in self.children:
            child.get_alice_measurements(self.alice.measurements)
            all_bases.append(child.bases)
        self.children[self.alice.get_index() - 1].set_flip()
        for child in self.children:
            child.get_all_bases(all_bases)
            child.gen_key()
            
    def __set_bases(self, qc: QuantumCircuit)->None:
        for i in range(self.no_of_children):
            self.children[i].gen_measurement_basis(qc, i + 1)

In [6]:
cs = CommonSpace(no_of_children = 3, no_of_measurements = 10)
cs.measurement_phase()
cs.agreement_phase()
print(cs.epsilon)

Job Status: job has successfully run
0.011969743075225917
